In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from directed_graph_generators.directed_stochastic_block_model import DirectedStochasticBlockModel

from helpers.wasserstein_distance import wasserstein_distance

In [2]:
#create a directed stochastic block model
n = 150
p = 0.5
q = 0.5
#3 x 3 block model
F = [[0.5,0.8,0.2],[0.2,0.5,0.8],[0.8,0.2,0.5]]
dsbm = DirectedStochasticBlockModel(k=3, N=n, p=p, q=q, F=F)
A = dsbm.adjacency_matrix

In [3]:
# for each pair of nodes, compute the shortest distance between them in the graph and store in array D
def shortest_path_length(A):
    n = A.shape[0]
    D = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            D[i,j] = np.inf
            if i == j:
                D[i,j] = 0
            if A[i,j] == 1:
                D[i,j] = 1
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if D[i,j] > D[i,k] + D[k,j]:
                    D[i,j] = D[i,k] + D[k,j]
    return D

D = shortest_path_length(A)

In [4]:
# with t =3, compute random walk distributions for P = D^(-1)A for each node
Deg = np.diag(np.sum(A, axis=1))
P = np.linalg.inv(Deg) @ A
P = np.linalg.matrix_power(P, 3)
random_walks = []
for i in range(n):
    random_walks.append(P[i,:].reshape(-1,1))

In [5]:
wasserstein_distances = np.zeros((n,n))
for i in range(n):
    for j in range(i,n):
        if i != j:
            wasserstein_distances[i,j] = wasserstein_distance(random_walks[i], random_walks[j], D)
            wasserstein_distances[i,j] = wasserstein_distances[i,j]

array([[0.        , 0.01923218, 0.02781231, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.02440416, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])